In [54]:
import os
import pandas as pd 
import numpy as np 
import subprocess
import math
from tqdm import tqdm 

import dataframe_image as dfi 

import matplotlib.pyplot as plt 
from matplotlib.patches import Patch
import matplotlib.cm 
import matplotlib.colors 
plt.rcParams['font.family'] = 'Arial'

from selenobot.files import FASTAFile, fasta_file_parser_gtdb, GBFFFile, BLASTFile
from selenobot.tools import BLAST
from selenobot.genes import Gene, Genome

%load_ext autoreload 
%autoreload 2

model_organisms = dict()
model_organisms['Pseudomonas aeruginosa'] = 'paer'
model_organisms['Escherichia coli'] = 'ecol'
model_organisms['Mycobacterium tuberculosis'] = 'mtub' 
model_organisms['Bacillus subtilis'] = 'bsub'
model_organisms['Aliivibrio fischeri'] = 'afis'

genome_metadata_df = pd.read_csv('../data/gtdb_genome_metadata_bacteria.csv', index_col=0)
genome_metadata_df = genome_metadata_df[genome_metadata_df.species.isin(model_organisms.keys())]
genome_metadata_df['code_name'] = genome_metadata_df.species.replace(model_organisms)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def make_summary_table(path='../data/model_organisms/summary_table.png'):   
    table_df = list()
    columns = ['ref. genome size', 'GTDB genome size', 'aligned sequences']
    for species, code_name in model_organisms.items():
        row = dict()
        row['species'] = species

        ref_fasta_file = FASTAFile(f'../data/model_organisms/ncbi_{code_name}_protein.faa')
        gtdb_fasta_file = FASTAFile(f'../data/model_organisms/gtdb_{code_name}_protein.faa')
        results_df = pd.read_csv(f'../data/model_organisms/{code_name}_results.csv', index_col=0)
        row['ref. genome size'] = len(ref_fasta_file)
        row['GTDB genome size'] = len(gtdb_fasta_file)
        row['aligned sequences'] = results_df.blast_possible_match.sum()
        table_df.append(row)
    table_df = pd.DataFrame(table_df).set_index('species')
    dfi.export(table_df, path)

make_summary_table()

In [3]:
def download_ncbi_data():
    '''Download the genomes, proteins, and GBFF files from NCBI for each model organism.'''
    genome_ids, code_names = list(genome_metadata_df.index), list(genome_metadata_df.code_name)

    cmd = 'datasets download genome accession {genome_ids} --filename ../data/model_organisms/ncbi.zip --include protein,gbff,genome'.format(genome_ids=' '.join(genome_ids))
    # subprocess.run(cmd, shell=True, check=True)
    # subprocess.run('unzip ../data/model_organisms/ncbi.zip', shell=True, check=True)
        
    for code_name, genome_id in zip(code_names, genome_ids):
        subprocess.run(f'cp ../data/model_organisms/ncbi/ncbi_dataset/data/{genome_id}/protein.faa ../data/model_organisms/ncbi_{code_name}_protein.faa', shell=True, check=True)
        subprocess.run(f'cp ../data/model_organisms/ncbi/ncbi_dataset/data/{genome_id}/genomic.gbff ../data/model_organisms/ncbi_{code_name}_genomic.gbff', shell=True, check=True)
        subprocess.run(f'cp ../data/model_organisms/ncbi/ncbi_dataset/data/{genome_id}/*_genomic.fna ../data/model_organisms/ncbi_{code_name}_genomic.fna', shell=True, check=True)


def run_blast():
    '''Run BLAST for the GTDB genome against the corresponding NCBI reference genome for each model organism.'''
    blast = BLAST()

    for species, code_name in model_organisms.items():
        query_path = f'../data/model_organisms/gtdb_{code_name}_protein.faa'
        subject_path = f'../data/model_organisms/ncbi_{code_name}_protein.faa'
        output_path = f'../data/model_organisms/gtdb_{code_name}_protein.blast.tsv'
        if not os.path.exists(output_path):
            print(f'Running BLAST for {species}')
            blast.run(query_path, subject_path, output_path=output_path, max_high_scoring_pairs=1, max_subject_sequences=1, make_database=False)


In [42]:
blast_cols = ['subject_id', 'sequence_identity', 'query_alignment_start', 'subject_alignment_start', 'query_sequence_length', 'subject_sequence_length', 'query_id']
gtdb_fasta_cols = ['start', 'stop', 'strand', 'partial', 'seq']
gbff_cols = ['gene', 'partial', 'strand', 'start', 'stop', 'copy_number', 'protein_id']

In [171]:
ncbi_fasta_df = FASTAFile('../data/model_organisms/ncbi_ecol_protein.faa').to_df()
gtdb_fasta_df = FASTAFile('../data/model_organisms/gtdb_ecol_protein.faa').to_df(parser=fasta_file_parser_gtdb)[gtdb_fasta_cols]

print('\nNumber of NCBI proteins:', len(ncbi_fasta_df))
print('Number of GTDB proteins:', len(gtdb_fasta_df))



Number of NCBI proteins: 4607
Number of GTDB proteins: 4717


In [ ]:
# Possible scenarios:
# (1) Sequence is a pseudogene. 
# (2) Sequence lies in an intergenic region. 
# (3) Sequence overlaps with an annotated protein-coding gene in the reference genome (error). 
# (5) Sequence is in the right frame, but the length is wrong (boundary error). 
# (6) Sequence is correct. 

# Also probably important to account for the size of the overlap. 

def search(df:pd.DataFrame, start:int=None, stop:int=None, strand:int=None, top_hit:bool=True):
    '''Look for sequences in the input GBFF file which overlap somewhere in the specified range.''' 
    def overlap(row):
        return len(np.intersect1d(np.arange(start, stop), np.arange(row.start, row.stop)))
    
    if (strand is not None):
        df = df[df.strand == strand]

    df = df.copy()
    df['overlap'] = df.apply(overlap, axis=1)
    df['percent_overlap'] = 100 * (df.overlap / (stop - start))
    df = df[df['overlap'] > 0]
    df = df.sort_values('overlap', ascending=False)
    return None if (len(df) == 0) else df.to_dict(orient='records')[0]


def find_matches(df:pd.DataFrame, code_name:str='ecol'):
    blast_df = BLASTFile(f'../data/model_organisms/gtdb_{code_name}_protein.blast.tsv').to_df()

    def is_match(hit) -> bool:
        if pd.isnull(hit.subject_id):
            return False
        if hit.sequence_identity < 95:
            return False  
        if not math.isclose(hit.subject_sequence_length, hit.query_sequence_length, abs_tol=5):
            return False
        return True

    df, blast_df = df.align(blast_df, axis=0, join='left')
    mask = blast_df.apply(is_match, axis=1)
    
    print(f'find_matches: Found {np.sum(mask)} sequences in the input genome which exactly match sequences in the reference.')
    return df[mask], df[~mask], blast_df[mask]


def find_boundary_errors(df:pd.DataFrame, code_name:str='ecol'):
    ''''''
    # NOTE: Seems worth distinguishing between left and right-side boundary errors. 
    blast_df = BLASTFile(f'../data/model_organisms/gtdb_{code_name}_protein.blast.tsv').to_df()

    def is_boundary_error(hit) -> bool:
        if pd.isnull(hit.subject_id):
            return False
        if hit.sequence_identity < 95:
            return False
        return True

    df, blast_df = df.align(blast_df, axis=0, join='left')
    mask = blast_df.apply(is_boundary_error, axis=1)

    info_df = blast_df.copy[mask]
    info_df['left_aligned'] = (info_df.query_alignment_start == 1) & (info_df.subject_alignment_start == 1)
    info_df['right_aligned'] = boundary_errors_info_df.query_alignment_end == boundary_errors_info_df.query_sequence_length - 1) & (boundary_errors_info_df.subject_alignment_end == boundary_errors_info_df.subject_sequence_length)

    print(f'find_boundary_errors: Found {np.sum(mask)} sequences in the input genome with boundary errors.')
    return df[mask], df[~mask], blast_df[mask] 


def find_pseudogenes(df:pd.DataFrame, code_name:str='ecol'):
    ''''''
    gbff_df = GBFFFile(f'../data/model_organisms/ncbi_{code_name}_genomic.gbff').to_df() 
    gbff_df = gbff_df[gbff_df.pseudo].drop(columns=['protein_id', 'seq'])

    def is_psuedogene(hit:dict, start:int=None, stop:int=None):
        if hit is None:
            return False, dict()
        if (hit['start'] == start) or (hit['stop'] == stop):
            return True, hit
        if hit['percent_overlap'] > 95:
            return True, hit
        return False, dict()

    info_df, mask = list(), list()
    for row in tqdm(list(df.itertuples()), desc='find_pseudogenes: Looking for overlapping pseudogenes in the reference genome...'):
        hit = search(gbff_df, start=int(row.start), stop=int(row.stop), strand=int(row.strand))
        val, hit = is_psuedogene(hit, start=row.start, stop=row.stop)
        hit['id'] = row.Index
        info_df.append(hit)
        mask.append(val)
    info_df = pd.DataFrame(info_df).set_index('id')
    mask = np.array(mask)

    print(f'find_pseudogenes: Found {np.sum(mask)} sequences in the input genome which match pseudogenes in the reference.')
    return df[mask], df[~mask], info_df[mask]


def find_intergenic(df:pd.DataFrame, code_name:str='ecol', allowed_overlap:int=30, include_hypothetical_proteins:bool=False):
    '''A GTDB ORF is intergenic if it either (1) does not overlap with any other nCDS element in the NCBI reference or (2) the overlap with a 
    protein (non-ppseudo) in the reference genome is no greater than the specified margin. 
    https://bmcgenomics.biomedcentral.com/articles/10.1186/1471-2164-15-721
    https://pmc.ncbi.nlm.nih.gov/articles/PMC525685/
    '''
    gbff_df = GBFFFile(f'../data/model_organisms/ncbi_{code_name}_genomic.gbff').to_df(drop_pseudogenes=True) 
    if (not include_hypothetical_proteins):
        gbff_df = gbff_df[gbff_df['product'].str.match('hypothetical protein')]

    def is_intergenic(hit:pd.DataFrame, allowed_overlap:int=20) -> (bool, dict):
        if hit is None:
            return True, dict()
        if hit['percent_overlap'] > 50:
            return False, hit
        if hit['overlap'] > allowed_overlap:
            return False, hit
        return True, dict()

    info_df, mask = list(), list()
    for row in tqdm(list(df.itertuples()), desc='find_intergenic: Looking for intergenic regions in the reference genome...'):
        hit = search(gbff_df, start=int(row.start), stop=int(row.stop))
        val, hit = is_intergenic(hit, allowed_overlap=allowed_overlap)
        hit['id'] = row.Index
        info_df.append(hit)
        mask.append(val)
    info_df = pd.DataFrame(info_df).set_index('id')
    mask = np.array(mask)

    print(f'find_intergenic: Found {np.sum(mask)} sequences in the input genome which are in intergenic regions.')
    return df[mask], df[~mask], info_df[~mask]


gtdb_fasta_df = FASTAFile('../data/model_organisms/gtdb_ecol_protein.faa').to_df(parser=fasta_file_parser_gtdb)[gtdb_fasta_cols]
matches_df, gtdb_fasta_df, _ = find_matches(gtdb_fasta_df)
boundary_errors_df, gtdb_fasta_df, boundary_errors_info_df = find_boundary_errors(gtdb_fasta_df)
pseudogenes_df, gtdb_fasta_df, _ = find_pseudogenes(gtdb_fasta_df)
intergenic_df, gtdb_fasta_df, _ = find_intergenic(gtdb_fasta_df)

find_matches: Found 4310 sequences in the input genome which exactly match sequences in the reference.
find_boundary_errors: Found 193 sequences in the input genome with boundary errors.
GBFFFile.__init__: Found 4818 coding sequences in the GBFF file.


GBFFFile.__init__: Parsing GBFF file entries.: 100%|██████████| 4818/4818 [00:00<00:00, 11063.47it/s]
find_pseudogenes: Looking for overlapping pseudogenes in the reference genome...: 100%|██████████| 214/214 [00:00<00:00, 233.98it/s]


find_pseudogenes: Found 123 sequences in the input genome which match pseudogenes in the reference.
GBFFFile.__init__: Found 4818 coding sequences in the GBFF file.


GBFFFile.__init__: Parsing GBFF file entries.: 100%|██████████| 4818/4818 [00:00<00:00, 10890.97it/s]


GBFFFile.to_df: Removing 199 pseudogenes from the GBFF file.


find_intergenic: Looking for intergenic regions in the reference genome...: 100%|██████████| 91/91 [00:00<00:00, 188.10it/s]

find_intergenic: Found 90 sequences in the input genome which are in intergenic regions.


In [175]:
gtdb_fasta_df

,start,stop,strand,partial,seq
id,,,,,
NZ_CP033092.2_2712,2859633,2859986,1,00,MPHTVHYPEQPSQGHVTPPPSRDAPVRFVLCCVAKTDNIRIYLVFH...


In [184]:
boundary_errors_info_df[(boundary_errors_info_df.query_alignment_start == 1) & (boundary_errors_info_df.subject_alignment_start == 1)]
boundary_errors_info_df[(boundary_errors_info_df.query_alignment_end == boundary_errors_info_df.query_sequence_length - 1) & (boundary_errors_info_df.subject_alignment_end == boundary_errors_info_df.subject_sequence_length)]

,query_id,subject_id,sequence_identity,alignment_length,mismatch,num_gap_openings,query_alignment_start,query_alignment_end,subject_alignment_start,subject_alignment_end,e_value,bit_score,query_coverage_per_subject,query_coverage_per_pair,query_sequence_length,subject_sequence_length
id,,,,,,,,,,,,,,,,
NZ_CP033092.2_12,NZ_CP033092.2_12,WP_000253469.1,100.000,430.0,0.0,0.0,1.0,430.0,16.0,445.0,0.000000e+00,862.0,99.0,99.0,431.0,445.0
NZ_CP033092.2_15,NZ_CP033092.2_15,WP_001332265.1,100.000,86.0,0.0,0.0,1.0,86.0,25.0,110.0,7.400000e-62,180.0,99.0,99.0,87.0,110.0
NZ_CP033092.2_36,NZ_CP033092.2_36,WP_011478308.1,100.000,396.0,0.0,0.0,17.0,412.0,1.0,396.0,0.000000e+00,776.0,96.0,96.0,413.0,396.0
NZ_CP033092.2_56,NZ_CP033092.2_56,WP_001070179.1,100.000,218.0,0.0,0.0,1.0,218.0,12.0,229.0,1.560000e-165,452.0,99.0,99.0,219.0,229.0
NZ_CP033092.2_145,NZ_CP033092.2_145,WP_001135732.1,100.000,50.0,0.0,0.0,21.0,70.0,1.0,50.0,1.600000e-32,103.0,70.0,70.0,71.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NZ_CP033091.2_97,NZ_CP033091.2_97,WP_042007484.1,99.187,123.0,1.0,0.0,1.0,123.0,31.0,153.0,3.180000e-87,247.0,99.0,99.0,124.0,153.0
NZ_CP033091.2_105,NZ_CP033091.2_105,WP_094096600.1,100.000,125.0,0.0,0.0,1.0,125.0,108.0,232.0,5.940000e-92,262.0,99.0,99.0,126.0,232.0
NZ_CP033091.2_109,NZ_CP033091.2_109,WP_012300440.1,100.000,141.0,0.0,0.0,9.0,149.0,1.0,141.0,4.510000e-101,283.0,94.0,94.0,150.0,141.0


In [151]:
gbff_df = GBFFFile('../data/model_organisms/ncbi_ecol_genomic.gbff').to_df(drop_pseudogenes=True) # Load in the GBFF file again, but keep the pseudogenes and duplicate proteins. 

pd.DataFrame(search(gbff_df, start=130629, stop=130925, top_hit=False))

GBFFFile.__init__: Found 4818 coding sequences in the GBFF file.


GBFFFile.__init__: Parsing GBFF file entries.: 100%|██████████| 4818/4818 [00:00<00:00, 11240.32it/s]


,strand,start,stop,partial,product,frameshifted,incomplete,internal_stop,protein_id,seq,overlap,percent_overlap
0,-1,130023,131519,00,L-xylulokinase,False,False,False,WP_000196108.1,MTQYWLGLDCGGSWLKAGLYDREGREAGVQRLPLCALSPQPGWAER...,296,100.0
